In [ ]:
# load csv and graph
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import numpy as np

# csv_file = 'results/test.csv'
# df = pd.read_csv(csv_file)

# graph the results

def plot_results(df, model_name):
    # Filter the DataFrame for the specific model
    model_df = df[df['model_name'] == model_name]

    # Convert confidence to numeric
    model_df['confidence'] = pd.to_numeric(model_df['confidence'], errors='coerce')

    # Create a bar plot for the confidence scores
    plt.figure(figsize=(12, 6))
    sns.countplot(data=model_df, x='answer', hue='confidence', palette='viridis')
    plt.title(f'Confidence Distribution for {model_name}')
    plt.xlabel('Answer')
    plt.ylabel('Count')
    plt.legend(title='Confidence', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
    # Save the plot
    plot_file = os.path.join('plots', f'{model_name}_confidence_distribution.png')
    os.makedirs(os.path.dirname(plot_file), exist_ok=True)
    plt.savefig(plot_file)
    print(f"Plot saved to {plot_file}")
    plt.close()

plot_results(df, 'openai/gpt-4o-mini')